In [16]:
import sqlite3
import json
from datetime import datetime

In [15]:
# with open ('F:/New folder/New folder (2)/reddit_data/{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:

In [29]:
timeframe = '2015-05'
sql_transaction = []
start_row = 0
cleanup = 1000000

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

In [4]:
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

In [18]:
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

In [19]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [20]:
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [21]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [22]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [23]:
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [24]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

In [30]:
# if __name__ == '__main__':
#     create_table()
#     row_counter = 0
#     paired_rows = 0

#     with open ('F:/New folder/New folder (2)/reddit_data/{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:
#         for row in f:
#             row_counter += 1
#             row = json.loads(row)
#             parent_id = row['parent_id']
#             body = format_data(row['body'])
#             created_utc = row['created_utc']
#             score = row['score']
#             comment_id = row['name']
#             subreddit = row['subreddit']
#             parent_data = find_parent(parent_id)
#             if score >= 2:
#                 existing_comment_score = find_existing_score(parent_id)
#                 if existing_comment_score:
#                     if score > existing_comment_score:
#                         if acceptable(body):
#                             sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
#                 else:
#                     if acceptable(body):
#                         if parent_data:
#                             sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
#                             paired_rows += 1
#                         else:
#                             sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
#             if row_counter % 100000 == 0:
#                 print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))
#             if row_counter > start_row:
#                 if row_counter % cleanup == 0:
#                     print("Cleanin up!")
#                     sql = "DELETE FROM parent_reply WHERE parent IS NULL"
#                     c.execute(sql)
#                     connection.commit()
#                     c.execute("VACUUM")
#                     connection.commit()

Total Rows Read: 100000, Paired Rows: 3220, Time: 2020-07-07 12:54:46.643728
Total Rows Read: 200000, Paired Rows: 8064, Time: 2020-07-07 12:57:38.807628
Total Rows Read: 300000, Paired Rows: 13710, Time: 2020-07-07 13:01:59.752195
Total Rows Read: 400000, Paired Rows: 19732, Time: 2020-07-07 13:06:01.484271
Total Rows Read: 500000, Paired Rows: 25657, Time: 2020-07-07 13:10:34.557118
Total Rows Read: 600000, Paired Rows: 31065, Time: 2020-07-07 13:23:19.914237
Total Rows Read: 700000, Paired Rows: 36074, Time: 2020-07-07 13:25:46.830563
Total Rows Read: 800000, Paired Rows: 41589, Time: 2020-07-07 13:27:40.218015
Total Rows Read: 900000, Paired Rows: 47722, Time: 2020-07-07 13:29:23.363336
Total Rows Read: 1000000, Paired Rows: 54085, Time: 2020-07-07 13:31:09.266293
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 58998, Time: 2020-07-07 13:32:33.761031
Total Rows Read: 1200000, Paired Rows: 64796, Time: 2020-07-07 13:33:49.748923
Total Rows Read: 1300000, Paired Rows: 70989, Time:

Total Rows Read: 10300000, Paired Rows: 594879, Time: 2020-07-07 15:43:38.690148
Total Rows Read: 10400000, Paired Rows: 601208, Time: 2020-07-07 15:44:52.276697
Total Rows Read: 10500000, Paired Rows: 607428, Time: 2020-07-07 15:45:56.849524
Total Rows Read: 10600000, Paired Rows: 613397, Time: 2020-07-07 15:47:02.809308
Total Rows Read: 10700000, Paired Rows: 619133, Time: 2020-07-07 15:48:15.644619
Total Rows Read: 10800000, Paired Rows: 625142, Time: 2020-07-07 15:49:29.847617
Total Rows Read: 10900000, Paired Rows: 631203, Time: 2020-07-07 15:50:37.240109
Total Rows Read: 11000000, Paired Rows: 637333, Time: 2020-07-07 15:51:51.439668
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 641852, Time: 2020-07-07 15:53:42.606249
Total Rows Read: 11200000, Paired Rows: 646756, Time: 2020-07-07 15:54:38.946764
Total Rows Read: 11300000, Paired Rows: 651968, Time: 2020-07-07 15:55:42.372736
Total Rows Read: 11400000, Paired Rows: 657833, Time: 2020-07-07 15:56:47.810717
Total Rows Read:

Total Rows Read: 20300000, Paired Rows: 1182304, Time: 2020-07-08 01:40:04.961669
Total Rows Read: 20400000, Paired Rows: 1188526, Time: 2020-07-08 01:41:59.726990
Total Rows Read: 20500000, Paired Rows: 1195060, Time: 2020-07-08 01:43:53.207683
Total Rows Read: 20600000, Paired Rows: 1201493, Time: 2020-07-08 01:46:10.338898
Total Rows Read: 20700000, Paired Rows: 1207838, Time: 2020-07-08 01:48:34.729887
Total Rows Read: 20800000, Paired Rows: 1214438, Time: 2020-07-08 01:51:00.848190
Total Rows Read: 20900000, Paired Rows: 1220959, Time: 2020-07-08 01:53:28.843104
Total Rows Read: 21000000, Paired Rows: 1227509, Time: 2020-07-08 01:55:55.181202
Cleanin up!
Total Rows Read: 21100000, Paired Rows: 1232241, Time: 2020-07-08 01:59:14.283298
Total Rows Read: 21200000, Paired Rows: 1237766, Time: 2020-07-08 02:01:06.764367
Total Rows Read: 21300000, Paired Rows: 1243288, Time: 2020-07-08 02:03:17.283195
Total Rows Read: 21400000, Paired Rows: 1249448, Time: 2020-07-08 02:05:39.780613
Tota

Total Rows Read: 30200000, Paired Rows: 1763381, Time: 2020-07-08 22:26:49.240156
Total Rows Read: 30300000, Paired Rows: 1768975, Time: 2020-07-08 22:28:31.086861
Total Rows Read: 30400000, Paired Rows: 1774498, Time: 2020-07-08 22:30:25.774955
Total Rows Read: 30500000, Paired Rows: 1779817, Time: 2020-07-08 22:32:09.387701
Total Rows Read: 30600000, Paired Rows: 1785806, Time: 2020-07-08 22:33:54.500312
Total Rows Read: 30700000, Paired Rows: 1792047, Time: 2020-07-08 22:35:39.201134
Total Rows Read: 30800000, Paired Rows: 1798253, Time: 2020-07-08 22:37:24.170036
Total Rows Read: 30900000, Paired Rows: 1804723, Time: 2020-07-08 22:39:16.543607
Total Rows Read: 31000000, Paired Rows: 1811310, Time: 2020-07-08 22:40:59.094015
Cleanin up!
Total Rows Read: 31100000, Paired Rows: 1816303, Time: 2020-07-08 22:44:35.642872
Total Rows Read: 31200000, Paired Rows: 1822549, Time: 2020-07-08 22:46:11.666378
Total Rows Read: 31300000, Paired Rows: 1828842, Time: 2020-07-08 22:47:57.360379
Tota

Total Rows Read: 40100000, Paired Rows: 2342923, Time: 2020-07-09 02:02:42.848118
Total Rows Read: 40200000, Paired Rows: 2348885, Time: 2020-07-09 02:05:06.605951
Total Rows Read: 40300000, Paired Rows: 2354998, Time: 2020-07-09 02:07:22.491539
Total Rows Read: 40400000, Paired Rows: 2361060, Time: 2020-07-09 02:09:10.044574
Total Rows Read: 40500000, Paired Rows: 2367437, Time: 2020-07-09 02:11:00.785010
Total Rows Read: 40600000, Paired Rows: 2373413, Time: 2020-07-09 02:13:33.362661
Total Rows Read: 40700000, Paired Rows: 2379148, Time: 2020-07-09 02:16:12.670311
Total Rows Read: 40800000, Paired Rows: 2384999, Time: 2020-07-09 02:19:01.463902
Total Rows Read: 40900000, Paired Rows: 2391256, Time: 2020-07-09 02:21:50.068277
Total Rows Read: 41000000, Paired Rows: 2396983, Time: 2020-07-09 02:25:02.656296
Cleanin up!
Total Rows Read: 41100000, Paired Rows: 2401029, Time: 2020-07-09 02:30:55.228174
Total Rows Read: 41200000, Paired Rows: 2405932, Time: 2020-07-09 02:33:27.874284
Tota

Total Rows Read: 50000000, Paired Rows: 2923526, Time: 2020-07-14 10:02:14.964799
Cleanin up!
Total Rows Read: 50100000, Paired Rows: 2927539, Time: 2020-07-14 10:09:26.089986
Total Rows Read: 50200000, Paired Rows: 2932554, Time: 2020-07-14 10:11:52.215458
Total Rows Read: 50300000, Paired Rows: 2938137, Time: 2020-07-14 10:14:39.063318
Total Rows Read: 50400000, Paired Rows: 2943991, Time: 2020-07-14 10:17:25.639333
Total Rows Read: 50500000, Paired Rows: 2950154, Time: 2020-07-14 10:20:14.238004
Total Rows Read: 50600000, Paired Rows: 2956240, Time: 2020-07-14 10:22:58.592954
Total Rows Read: 50700000, Paired Rows: 2962150, Time: 2020-07-14 10:25:40.282473
Total Rows Read: 50800000, Paired Rows: 2968184, Time: 2020-07-14 10:28:31.538228
Total Rows Read: 50900000, Paired Rows: 2974243, Time: 2020-07-14 10:31:56.490934
Total Rows Read: 51000000, Paired Rows: 2980365, Time: 2020-07-14 10:36:34.784510
Cleanin up!
Total Rows Read: 51100000, Paired Rows: 2985194, Time: 2020-07-14 10:45:26

In [31]:
a = 5
a 

5

### Training

In [3]:
import sqlite3
import pandas as pd

timeframes = ['2015-05']

In [4]:
for timeframe in timeframes:
    connection = sqlite3.connect('{}.db'.format(timeframe))
    c = connection.cursor()
    limit = 5000
    last_unix = 0
    cur_length = limit
    counter = 0
    test_done = False

    while cur_length == limit:

        df = pd.read_sql("SELECT * FROM parent_reply WHERE unix > {} and parent NOT NULL and score > 0 ORDER BY unix ASC LIMIT {}".format(last_unix,limit),connection)
        last_unix = df.tail(1)['unix'].values[0]
        cur_length = len(df)

        if not test_done:
            with open('test.from','a', encoding='utf8') as f:
                for content in df['parent'].values:
                    f.write(content+'\n')

            with open('test.to','a', encoding='utf8') as f:
                for content in df['comment'].values:
                    f.write(str(content)+'\n')

            test_done = True

        else:
            with open('train.from','a', encoding='utf8') as f:
                for content in df['parent'].values:
                    f.write(content+'\n')

            with open('train.to','a', encoding='utf8') as f:
                for content in df['comment'].values:
                    f.write(str(content)+'\n')

        counter += 1
        if counter % 20 == 0:
            print(counter*limit,'rows completed so far')

100000 rows completed so far
200000 rows completed so far
300000 rows completed so far
400000 rows completed so far
500000 rows completed so far
600000 rows completed so far
700000 rows completed so far
800000 rows completed so far
900000 rows completed so far
1000000 rows completed so far
1100000 rows completed so far
1200000 rows completed so far
1300000 rows completed so far
1400000 rows completed so far
1500000 rows completed so far
1600000 rows completed so far
1700000 rows completed so far
1800000 rows completed so far
1900000 rows completed so far
2000000 rows completed so far
2100000 rows completed so far
2200000 rows completed so far
2300000 rows completed so far
2400000 rows completed so far
2500000 rows completed so far
2600000 rows completed so far
2700000 rows completed so far
2800000 rows completed so far
2900000 rows completed so far
3000000 rows completed so far
3100000 rows completed so far
